# Estabilidad y condicionamiento

## 1. Estabilidad numérica

De manera general, puede decirse que cuando un proceso numérico es inestable los errores de cada etapa del proceso se acumulan o amplifican en etapas posteriores, afectando de forma drástica a la precisión del cálculo.
También podemos pensar en la *estabilidad* como la propiedad que tienen los algoritmos que consiguen resultados con un buen grado de precisión si parten de datos próximos a los reales.
En el contexto de este curso, es suficiente con conocer el concepto anterior y saber identificar cuándo un algoritmo es o no estable.

**Ejercicio 1 -** Considera dos algoritmos que pretenden calcular el valor de $7^{-100}$. Cada uno de ellos es una aplicación particular de dos algoritmos más generales que sirven para calcular las potencias sucesivas de un número positivo $\lambda < 0$:
$$
\left\{ \begin{array}{ll} 
x_0 &= 1 \\
x_n &= \lambda x_{n-1}
\end{array} \right. 
\quad \textrm{y} \quad 
\left\{ \begin{array}{ll} 
x_0 &= 1 \\
x_1 &= \lambda \\
x_{n} &= \left( 3 + \lambda \right) x_{n-1} - 3 \lambda x_{n-2}
\end{array} \right. 
$$
En ambos casos, el término general de la sucesión $\left\{ x_n \right\}^{\infty}$ es $\lambda^n$. Calcula el término $x_{100}$ usando ambos algoritmos e interpreta los resultados.

<!--
```
# Este primer enfoque es muy lento
def a1(n: int, lamb: float) -> float:
    if n == 0:
        return 1
    return lamb * a1(n - 1, lamb)

def a2(n: int, lamb: float) -> float:
    if n == 0:
        return 1
    elif n == 1:
        return lamb
    return (3.0 + lamb) * a2(n - 1, lamb) - 3 * lamb * a2(n - 2, lamb)

lamb = 1.0 / 7.0
#for n in range(101):
#    print(f"{n=}  x_n (alg1)={a1(n, lamb)}  x_n (alg2)={a2(n, lamb)} ")

#####################################################

lamb = 1.0 / 7.0
x2a1 = 1
x2a2 = 1
x1a1 = lamb * x2a1
x1a2 = lamb

print(f"n=0  x0a1={x2a1:.13E}  x0a1={x2a2:.13E}")
print(f"n=1  x0a1={x1a1:.13E}  x0a1={x1a2:.13E}")

for n in range(2, 101):
    x0a1 = lamb * x1a1
    x0a2 = (3.0 + lamb) * x1a2 - 3 * lamb * x2a2

    if n % 7 == 0:
        print(f"{n=}  {x0a1=:.13E}  {x0a2=:.13E} ")

    x2a1 = x1a1
    x2a2 = x1a2
    x1a1 = x0a1
    x1a2 = x0a2

```
-->

In [4]:
print(f'valor a calcular = {(1/7) ** 100}')

valor a calcular = 3.0916904080902036e-85


In [13]:
def alg1(labm : float, n_iter: int) -> float:

    x_nold = 1.0 #xn-1
    
    for i in range(n_iter):
        x_n = x_nold * lamb
        x_nold = x_n

    return x_n


def alg2(labm : float, n_iter: int):

    x_nold = lamb
    x_nold_old = 1.0 #xn-2

    for i in range(n_iter):
        x_n = (3.0 + lamb) * x_nold - 3 * lamb * x_nold_old

        x_nold_old = x_nold 
        x_nold = x_n
    
    return x_n



lamb = 1.0 / 7.0
print(f' el valor del priemr algotrimo es {alg1(lamb, 100)}')
print(f' el valor del priemr algotrimo es {alg2(lamb, 100)}')


 el valor del priemr algotrimo es 3.091690408090202e-85
 el valor del priemr algotrimo es -5.891118499898032e+30


## 2. Condicionamiento

Diremos que un problema está *mal condicionado* cuando pequeños cambios en los datos dan lugar a grandes cambios en los resultados. Las técnicas que se emplean en el estudio del condicionamiento de un problema están fuertemente ligadas a la estructura del mismo. En general, a la hora de resolver un problema $y = f(x)$ se intenta definir un número de condición $\kappa = \kappa (x) \ge 0$ de forma que:

$$
\frac{\|f(\widetilde{x})-f(x)\|}{\|f(x)\|} \simeq \kappa(x) \frac{\|\tilde{x}-x\|}{\|x\|}
$$

<span style="color: orange"> pone una norma porque a la hora de hacerlo en mas dimensiones el b¡valor absoluto se hace con una norma</span>.

Este número $\kappa$ indicará si el problema está bien o mal condicionado, según sea próximo o no a 1. Si el número de condición es menor que 1 o está próximo a 1, el error del dato no se amplificará mucho y el error del resultado será, a lo sumo, del mismo orden que el error en el dato. Por el contrario, si este número de condición toma valores muy grandes, el error del resultado final podría sufrir una gran ampliación respecto del error en el dato.

Como se ha mencionado anteriormente, el estudio del condicionamiento está muy ligado a cada aplicación particular. En el contexto de este curso, estudiaremos solamente dos casos: la evaluación de una función diferenciable y la resolución de sistemas lineales.


### 2.1 Condicionamiento en la evaluación de una función diferenciable
Consideremos la evaluación de una función real de variable real $f: \mathbb{R} \rightarrow \mathbb{R}$ en $x$. Si en lugar de $x$ tomamos una aproximación suya $\widetilde{x}$ con $ \left| x - \widetilde{x} \right| \ll 1$ (por ejemplo, su redondeo), el teorema del Valor Medio asegura que
$$
f(\widetilde{x})-f(x)=f^{\prime}(\xi)(\widetilde{x}-x) \simeq f^{\prime}(x)(\widetilde{x}-x)
$$
De esta forma, si $f'(x)$ no es muy grande, el efecto de la perturbación sobre $f(x)$ es pequeño. Concretamente, el error relativo de la perturbación viene dado por
$$
\left|\frac{f(\widetilde{x})-f(x)}{f(x)}\right| \simeq\left|\frac{f^{\prime}(x)}{f(x)}\right||\widetilde{x}-x|= \left|\frac{f^{\prime}(x)}{f(x)} x\right| \cdot \left|\frac{\tilde{x}-x}{x}\right|
$$
donde el primer factor del lado derecho de esta ecuación es el **número de condición** $\kappa(x) = \left|\frac{f^{\prime}(x)}{f(x)} x\right|$, y el segundo el error relativo de la entrada.

**Ejercicio 2 -** Considera la función $f(x) = 1 - \sqrt{1 - x^2}$ para $-1 \le x \le 1$. Calcula el número de condición $\kappa (x)$ para estudiar en qué puntos puede presentar problemas de evaluación.

Una forma práctica de interpretar el número de condición es la siguiente: nos da información del número de cifras significativas que se van a perder al resolver el problema. Por ejemplo, si al resolver un problema con un número de condición $\kappa = 2000$, trabajando con precisión doble (error de máquina  $\varepsilon = 2.22 \cdot 10^{-16}$), el resultado tendrán una precisión de $2000 \cdot 2.22 \cdot 10^{-16} = 4.44e-13 < 0.5 \cdot 10^{-12} \Rightarrow$ 12 
cifras significativas.

<span style="color: orange"> en precision doble el error es de 2 * 10^-16, entonces si tu k chiquita tiene un error de 10^6 sabemos por defecto que elerror va a ser de 10^-10</span>
<span style="color: orange"></span>

In [1]:
import numpy as np
import matplotlib.pyplot as plt 


def f(x: float):
    return 1.0 - (1.0 - x ** 2) ** 0.5

def df(x: float):
    return x / (1.0 - x**2) ** 0.5

def kappa(x):
    return abs(x * df(x) / f(x))


x = np.linspace(-1, 1, 100) # particion

y_kappa = np.array([kappa(i) for i in x]) # te esta dando el f(x) para cada valro de la particion x


plt.plot(x, y_kappa)


print(f"n1 de condicion en x = 0.99999999999  = {kappa(0.999999999999)}")





ModuleNotFoundError: No module named 'matplotlib'


### 2.2 Condicionamiento en la resolución de sistemas lineales

A continuación estudiaremos cómo definir el condicionamiento en un sistema lineal de la forma $Ax=b$, (<span style="color: orange"> la Ax no va a ser la entrada sino la salida </span>) siendo $A$ la matriz de coeficientes (invertible), $x$ el vector de incógnitas y $b$ el vector de términos independientes. En el supuesto de que se tome como segundo miembro en lugar del vector $b$ una perturbación de este $b + \Delta b$, la solución del problema ahora será $x + \Delta x $, y se verifica entonces que:
$$
A \left( x + \Delta x \right) = b + \Delta b \Rightarrow A \Delta x = \Delta b \Rightarrow \Delta x = A^{-1} \Delta b
$$
<span style="color: orange"> el Ax = b pot lo que en ambos lados de la ecuacion podemos simplificarlo y por eso solo queda Adeltax = deltab</span>

Tomando normas y utilizando la desigualdad triangular:
$$
\left\| \Delta x \right\| \le \left\| A^{-1} \right\| \cdot \left\| \Delta b \right\|
$$
Por otra parte, 
$$
Ax=b \Rightarrow \left\| b \right\| \le \left\| A \right\| \cdot \left\| x \right\| \Rightarrow \frac{1}{\left\| x \right\| } \le \frac{\left\| A \right\|}{\left\| b \right\|}
$$

y se llega por tanto a:
$$
\frac{\left\| \Delta x \right\|}{\left\| x \right\|} \le \left\| A \right\| \cdot \left\| A^{-1} \right\| \cdot \frac{\left\| \Delta b \right\|}{\left\| b 
\right\|}
$$
de donde se deduce que la cantidad $\left\| A \right\| \cdot \left\| A^{-1} \right\|$ es el número de condición para sistemas lineales, también denotado como $\kappa \left( A \right)$. Si en lugar de una pequeña variación en $b$ se supusiese una pequeña variación en $A$, se llega un resultado análogo, con el mismo número de condición (ver demostración por ejemplo en *Introduction to Numerical Linear Algebra and Optimisation, P. G. Ciarlet*)

<span style="color: orange">no depende del vector de coeficientes independientes (x) pero si de la madriz A</span>


Para completar el resultado anterior, falta recordar el concepto de norma matricial. Nótese que las normas matriciales y vectoriales escogidas para las anteriores expresiones deben ser coherentes (deben estar *subordinadas*). Recuérdese las siguientes tres normas para una matriz $A$ de dimensiones $n \times m$:
 - $\left\| A \right\|_1 = \max_{j=1,\dots,n} \sum_{i=1}^m \left|A_{ij}\right|$, subordinada a la norma vectorial $\ell_1$ ([norma Manhattan](https://en.wikipedia.org/wiki/Norm_(mathematics)#Taxicab_norm_or_Manhattan_norm)).
 - $\left\| A \right\|_2 = \max_{k=1,\dots,n} \sqrt{\mu_i}$, subordinada a la norma vectorial $\ell_2$ ([norma euclídea](https://en.wikipedia.org/wiki/Norm_(mathematics)#Euclidean_norm))
 - $\left\| A \right\|_\infty = \max_{i=1,\dots,m} \sum_{j=1}^n \left|A_{ij}\right|$, subordinada a la norma vectorial $\ell_\infty$ ([norma infinito](https://en.wikipedia.org/wiki/Uniform_norm)).
  
siendo $\mu_i$ los autovalores de la matriz $A^T A$. Para denotar de forma inequívoca qué norma se está utilizando para calcular el número de condición, en ocasiones se le añade un subíndice, por ejemplo  $\kappa_2$. 

En el contexto del estudio del condicionamiento, hay un resultado interesante que es de gran utilidad (ver demostración por ejemplo en *Matrix Perturbation Theory, G. W. Stewart, J.-G. Sun*). Si $A$ es una [matriz normal](https://en.wikipedia.org/wiki/Normal_matrix) (es decir, cumple que $A^TA=AA^T$) usando la norma $\left\| \cdot \right\|_2$, el número de condición $\kappa \left( A \right)$ de una determinada matriz puede escribirse como:
$$
\kappa \left( A \right) = \frac{\left| \lambda_{\mathrm{max}} \right|}{\left| \lambda_{\mathrm{min}} \right|}
$$
siendo $\lambda_{\mathrm{max}}$ y $\lambda_{\mathrm{min}}$ los autovalores máximo y mínimo de $A$, respectivamente.


**Ejercicio 3 -** Analiza si los siguientes sistemas de ecuaciones son o no sensibles a la variación de los datos de entrada. Primero haz una exploración experimentando en Python con los datos, y luego realiza una deducción analítica para justificar el comportamiento observado:

$$
\left\{\begin{array}{ll}
      x + 2y &= 4 \\
      2x + 3.999y &= 7.999 \\
\end{array} \right. 
\qquad
\left\{\begin{array}{ll}
      x + 2y &= 4 \\
      2x + 3y &= 7 \\
\end{array} \right. 
$$

<span style="color: orange"> si un sistema esta  mal condicionada es porque al sumarle una variacion muy pequeña la solución tiene un error significativo mayor del esperado</span>

In [1]:
#te estan pidiendo que analices el condicionamiento
import numpy as np
A1 = np.array([[1 , 2], [2, 3.999]])
b1 = np.array([4, 3.999])
A2 = np.array([[1, 2], [2, 3]])
b2 = np.array([4, 7])

A1_inv = np.linalg.inv(A1)
A2_inv = np.linalg.inv(A2)


eig1 = np.linalg.eigvals(A1) #autovalores = eig
eig2 = np.linalg.eigvals(A2)


cond1_norm1 = np.linalg.norm(A1_inv, 1) * np.linalg.norm(A1, 1) # np.linalg.norm(A1, 1) -> calcular al norma de (matriz, el tipo de norma)
cond2_norm1 = np.linalg.norm(A2_inv, 1) * np.linalg.norm(A2, 1)

cond1_norm2 = np.linalg.norm(A1_inv, 1) * np.linalg.norm(A1, 2)
cond2_norm2 = np.linalg.norm(A2_inv, 1) * np.linalg.norm(A2, 2)

cond1_norm_inf = np.linalg.norm(A1_inv, np.inf) * np.linalg.norm(A1, np.inf)
cond2_norm_inf = np.linalg.norm(A2_inv, np.inf) * np.linalg.norm(A2, np.inf)


print(f'{cond1_norm1 = }, {cond2_norm1 = } ')
print(f'{cond1_norm2 = }, {cond2_norm2 = } ')
print(f'{cond1_norm_inf = }, {cond2_norm_inf = } ')

cond1_norm1 = np.float64(35988.00100000397), cond2_norm1 = np.float64(25.0) 
cond1_norm2 = np.float64(29990.200991994334), cond2_norm2 = np.float64(21.180339887498945) 
cond1_norm_inf = np.float64(35988.00100000397), cond2_norm_inf = np.float64(25.0) 


In [3]:
print(A1)
print(A1_inv)

[[1.    2.   ]
 [2.    3.999]]
[[-3999.  2000.]
 [ 2000. -1000.]]


In [9]:
print(f' es la matriz A1 normal {np.allclose(A1 * A1.T, A1.T * A1)}')
print(f' es la matriz A2 normal {np.allclose(A2 * A2.T, A2.T * A2)}')

 es la matriz A1 normal True
 es la matriz A2 normal True


In [10]:
print(abs(max(eig1)) / abs(min(eig1)))
print(abs(max(eig2)) / abs(min(eig2)))

24992.000959963632
17.944271909999152


In [11]:
from random import random

delta = random() /100
delta

0.007839052198661225

In [14]:
print(np.linalg.solve(A1, b1))
print(np.linalg.solve(A1 + delta, b1))
print()
print(np.linalg.solve(A2, b2))
print(np.linalg.solve(A2 + delta, b2))

[-7998.  4001.]
[1170.80343821 -585.69506664]

[2. 1.]
[2.02351716 0.97648284]


In [18]:
err_rel_entrada1 = delta / np.linalg.norm(A1)
err_rel_salida1 = np.linalg.norm(np.linalg.solve(A1, b1) - np.linalg.solve(A1 + delta, b1)) / np.linalg.norm(np.linalg.solve(A1, b1))

print(err_rel_entrada1)
print(err_rel_salida1)

0.0015680613182513172
1.1463870552231634
